<a href="https://colab.research.google.com/github/smeenakshi1997/GRIHAM-C_OGD/blob/main/GRIHAMC_ReportGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **GRIHAM-C Report Generator code**

*** This code is for demonstration purposes and generates the classification_report.json file, one id at a time. This code can be modified slightly and automated completely on the backend of the existing Grievance submission portal.**

*** Consists of a separate translation generation code which can be utilised for other language grievance classification. Refer to the attached files and README in github**

* All the code in this system is developed and tested for demonstartion purposes only and is developed keeping in mind of extensions further with simple executions that do not require any alterations to the functioning of the clustering system itself.
* All the code in this software named GRIHAM-C has been independantly developed, tested and presented by me. Unauthorised use of this software is prohibited.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade TensorFlow

In [ ]:
!pip install transformers  #==4.37.1
!pip install torch   #==2.2.0
!pip install tqdm
!pip install scikit-learn

In [ ]:
# Make sure to check the version of transformers and torch while training the model. Re-install if the versions dont match by un-commenting the verions in the previous cell.
import transformers
import torch
print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)

Transformers version: 4.37.2
Torch version: 2.1.0+cu121


In [ ]:
import pickle
import torch
from transformers import pipeline, TFBartForSequenceClassification, BartTokenizer

# Load the fine-tuned model and tokenizer
loaded_model = TFBartForSequenceClassification.from_pretrained('/content/problem_statement_1_and_2/cluster_model_worksfine')
loaded_tokenizer = BartTokenizer.from_pretrained('/content/problem_statement_1_and_2/cluster_model_worksfine')

map_location=torch.device('cpu')

classifier = pipeline("zero-shot-classification", model=loaded_model, tokenizer=loaded_tokenizer)

All PyTorch model weights were used when initializing TFBartForSequenceClassification.

All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.


Use the code below for the pickle file of the same model on an GPU machine

In [ ]:
# Can be used with the finetuned model as pickle file
# Load the fine-tuned model and tokenizer from the pickle file
#with open('/content/drive/MyDrive/problem_statement_1_and_2/cluster_model_worksfine.pkl', 'rb') as file:
#     saved_model_data = pickle.load(file)

#loaded_model = saved_model_data['model']
#loaded_tokenizer = saved_model_data['tokenizer']

#loaded_model.to('cpu')

#classifier = pipeline("zero-shot-classification", model=loaded_model, tokenizer=loaded_tokenizer)

In [ ]:
def classification (text, stagelabels):
    top5 = []
    candidate_labels = stagelabels #catdf['stage2'] = ['Direct Taxes', 'PAN issues', 'Establishment issues']
    # Tokenize the input string
    tokenizer = loaded_tokenizer

    tokenized_input = tokenizer(text, return_tensors="pt")
    # Now `tokenized_input` contains the necessary tokens for the model

    output = classifier(text, candidate_labels, multi_label=True)

    if (len(output['labels'])) > 5:
       for m in range (0,4):
          if output['labels']:
             top5.append(output['labels'][m])
    elif (len(output['labels'])) <= 5 and (len(output['labels'])) != 0:
         top5 = output['labels']
    else:
         top5 = []

    return (output['labels'][0], top5)


In [ ]:
def classify_items(json_data, org_code, sequence_to_classify): # instead of org_code stage 3 description can be given
    # Convert JSON data to a list of dictionaries
    data = json.loads(json_data)

    if org_code.startswith("GOV"):
       data = data
       original_data = data.copy()
    else:
       # Filter data by OrgCode
       data = [item for item in data if item['OrgCode'] == org_code]
       original_data = data.copy()
       # data = [item for item in data if item['Parent'] == (item ['Code'] if item['Description'] == stage3_description)] i.e. find the code of the stage3 item description

    # Initialize result dictionary
    result = {}

    text = sequence_to_classify

    for stage in range(1, 9): # Alter the stage no according to the stage 3 description
        # Get items for the current stage
        stage_items = [item for item in data if item['Stage'] == stage]
        #print(stage_items)

        # If there are no items for the current stage, break the loop
        if not stage_items:
            break

        # Get the descriptions of the items
        descriptions = [item['Description'] for item in stage_items] #DescriptionHindi

        # Classify the descriptions
        best_item, top_5_items = classification(text, descriptions)

        # Add the best item to the result
        result[f'Stage {stage}'] = {
            'Best Item': best_item,
            'Top 5 Items': top_5_items
        }

        #print(best_item)
        with open('result.json', 'w') as f:
             json.dump(result, f)

        for item in data:
            if item['Description'] == best_item:
               best_item_code = item['Code']
               break
        result[f'Stage {stage}'].update({"Best Item Code":best_item_code})
        # Update data to only include items whose parent is the code of the best item
        data = [item for item in original_data if item['Parent'] == best_item_code]
        #print(data)
        #print(output)

    return result

After running the following cell, enter the Grievance code and check the language of the text submitted.

In [ ]:
import json

with open('/content/problem_statement_1_and_2/no_pii_grievance.json', 'r') as f:
     grievance_df = json.load(f)

with open('/content/problem_statement_1_and_2/CategoryCode_Mapping_v2.json', 'r') as c:
     # Load JSON data from file
     json_dat = json.load(c)    # Your JSON data

json_data = json.dumps(json_dat)
result = []

gdf = {"code":[], "text":[]}

gr_id =  input("Enter the Grievance Id: ")#"MEAPD/E/2023/0000002" #"GOVUP/E/2023/0000051" #"CBODT/E/2023/0000001" #"MOLBR/E/2023/0000004" #"MORLY/E/2023/0000001" #"MOLBR/E/2023/0000003"
for id in grievance_df:
    if id['_id'] == gr_id:
       gdf["code"] = id['org_code']
       gdf["text"] = id['subject_content_text']
       break

#id = 0 #Id is the only needed to be given usually the unique id of the grievance.the id will fetch all the details from the DB including org code

sequence_to_classify = gdf['text'] #Text to classify
org_code = gdf['code'] #OrgCode

print(sequence_to_classify)


***Check the output to check whether the string is in English or other language. If in other language then translate it to English, from the cell below after choosing the input language.***

In [ ]:
result = classify_items(json_data, org_code, sequence_to_classify)

print(result)
print(len(result))

In [ ]:
best_items_dict = {'Best Items': {}}

for stage, values in result.items():
    best_item = values['Best Item']
    best_items_dict['Best Items'][f'{stage}'] = best_item


print(best_items_dict)

{'Best Items': {'Stage 1': 'Labour and Employment', 'Stage 2': 'Transfer related issues', 'Stage 3': 'Transfer in/out/Form 13/ Online transfer related issues'}}


In [ ]:
print(sequence_to_classify)
print(org_code)

Labour and Employment >> Transfer related issues >> Transfer in/out/Form 13/ Online transfer related issues

Name and Address of Establishment : PERSOLKELLY INDIA PRIVATE LIMITED
UAN No. : X0X3X9X5X8X0
PF Code/ PF Account No. : X-X-X-X-X
PPO No. : Not Provided
Scheme Certificate Number : Not Provided
PF Office : Regional Office, Bandra 1
-----------------------
Hello dear sir madam
My name is SYED SHEBAAZ AHMED
My uan number is X0X3X9X5X8X0
My pf number is X-X-X-X-X
My company name is PERSOLKELLY INDIA PRIVATE LIMITED
Hers about company details

Sir i apply for transfer request but epfo still rejected reason is Break statement
Rejected reason: Your Claim  Claim Id - X-X-X-X-X  has been rejected due to : 1) PAYMENT NOT RECEIVED FOR THE MONTH OF 05/2022 CLARIFY THE SAME 

So i contact to my Hr he&#39;s given to me Clearfication Letter break statment and also sending Epfo office regarding
So i request you to Epfo department please approve my transfer
Please find the attachment pdf file
Pl

## **Summarizer system for Report Generation**



In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the fine-tuned model and tokenizer
files = '/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer'

model = BartForConditionalGeneration.from_pretrained(files)
tokenizer = BartTokenizer.from_pretrained(files)

# Example text for summarization
input_text = sequence_to_classify
# Tokenize and generate summary
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=False)
summary_ids = model.generate(**inputs)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Generated Summary:", summary)

Generated Summary: Labour and Employment >> Transfer related issues >> Transfer in/out/Form 13/ Online transfer related issues. Name and Address of Establishment : PERSOLKELLY INDIA PRIVATE LIMITED. Uan No. : X0X3X9X5X8X0.PF Code/ PF Account No.  : X-X-X X-Z.Scheme Certificate Number : Not Provided.


## **Generate Report as JSON file as soon as the grievance is received**

In [ ]:
import json

# Load existing JSON file
existing_file_path = '/content/problem_statement_1_and_2/classification_report.json'
try:
    with open(existing_file_path, 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, start with an empty list
    data = []

# Create a new JSON object to add
classification_report = {
    "grievance_id": gr_id,
    "org_code": org_code,
    "grievance_text": sequence_to_classify,
    "classification_result": result,
    "best_items_dict": best_items_dict,
    "summary_result": summary,
    "remarks_text": '',
    "translated_text": ''
}
# Append the new JSON object to the existing array
data.append(classification_report)

# Write the updated data back to the JSON file
with open(existing_file_path, 'w') as file:
    json.dump(data, file, indent=2)

print("JSON object of classication added successfully and classification_report.json generated!")

JSON object of classication added successfully and classification_report.json generated!


## **Additional Independent Translator for Indian Langauges (Except Odia): Use this cell to translate the sequence_to_classify for other language**
Use the cells below to continue....

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

#article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"

article = sequence_to_classify

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

lang_list = ["hi_IN", "ta_IN", "ml_IN" , "mr_IN", "bn_IN", "te_IN"]

tokenizer.src_lang = "hi_IN"
encoded_hi = tokenizer(article, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

print(translation)

#sequence_to_classify = translation

['Ayush >> Miscellaneous >> Others ----------------------- In service, Mr. Pranam, Subject: - In relation to an important information, Mr., I am a resident of Maharaja Dube, Gram - Hasanpurwa, Siwan (Bihar, 841286). I have enrolled in the DYT course “Mr. Padnjali Institute of Yoga and Natural Medicine (MPYPCS)” Chhapra, (Bihar, 841301). The people of this institution say that these institutions are recognised by the Yoga Certification Board (YCB), Indian Yoga Association (IYA), Ministry of Ayush (MoY) and UGC. Sir, is this institution recognised by all the above mentioned institutions and will its degree be recognised across India? Please help me and guide me as I have yet to take a graduate degree in Yoga. Please help. Thank you! Waiting for']


In [ ]:
#translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

print("Translated Text: ", translation[0])

sequence_to_classify_trans = translation[0]

#sequence_to_classify = translation

Translated Text:  Ayush >> Miscellaneous >> Others ----------------------- In service, Mr. Pranam, Subject: - In relation to an important information, Mr., I am a resident of Maharaja Dube, Gram - Hasanpurwa, Siwan (Bihar, 841286). I have enrolled in the DYT course “Mr. Padnjali Institute of Yoga and Natural Medicine (MPYPCS)” Chhapra, (Bihar, 841301). The people of this institution say that these institutions are recognised by the Yoga Certification Board (YCB), Indian Yoga Association (IYA), Ministry of Ayush (MoY) and UGC. Sir, is this institution recognised by all the above mentioned institutions and will its degree be recognised across India? Please help me and guide me as I have yet to take a graduate degree in Yoga. Please help. Thank you! Waiting for


# **Classification for translated text**

In [ ]:
result = classify_items(json_data, org_code, sequence_to_classify_trans)

print(result)
print(len(result))

{'Stage 1': {'Best Item': 'Ayush', 'Top 5 Items': ['Ayush'], 'Best Item Code': 22801}, 'Stage 2': {'Best Item': 'Miscellaneous', 'Top 5 Items': ['Miscellaneous', 'Ayush Schemes', 'Ayush Education in National Institutes', 'Ayush Research'], 'Best Item Code': 23194}, 'Stage 3': {'Best Item': 'Others', 'Top 5 Items': ['Others', 'All Sections of the MoA', 'All field Organizations of the MoA'], 'Best Item Code': 23197}}
3


In [ ]:
best_items_dict = {'Best Items': {}}

for stage, values in result.items():
    best_item = values['Best Item']
    best_items_dict['Best Items'][f'{stage}'] = best_item


print(best_items_dict)

{'Best Items': {'Stage 1': 'Ayush', 'Stage 2': 'Miscellaneous', 'Stage 3': 'Others'}}


# **Summary for translated Text**

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the fine-tuned model and tokenizer
files = '/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer'

model = BartForConditionalGeneration.from_pretrained(files)
tokenizer = BartTokenizer.from_pretrained(files)

# Example text for summarization
input_text = sequence_to_classify_trans
# Tokenize and generate summary
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=False)
summary_ids = model.generate(**inputs)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Generated Summary:", summary)

Generated Summary: Mr. Pranam is a resident of Maharaja Dube, Gram - Hasanpurwa, Siwan (Bihar, 841286) He has enrolled in the DYT course “Mr. Padnjali Institute of Yoga and Natural Medicine (MPYPCS)” Chhapra, (B Bihar, 841301) The people of this institution say that these institutions are recognised by the Yoga Certification Board.


# **Report for Translated text**

In [ ]:
import json

# Load existing JSON file
existing_file_path = '/content/classification_report_hindi.json' #'/content/classification_report_hindi.json'
try:
    with open(existing_file_path, 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, start with an empty list
    data = []

# Create a new JSON object to add
classification_report = {
    "grievance_id": gr_id,
    "org_code": org_code,
    "grievance_text": sequence_to_classify,
    "classification_result": result,
    "best_items_dict": best_items_dict,
    "summary_result": summary,
    "remarks_text": '',
    "translated_text": sequence_to_classify_trans
}
# Append the new JSON object to the existing array
data.append(classification_report)

# Write the updated data back to the JSON file
with open(existing_file_path, 'w') as file:
    json.dump(data, file, indent=2)

print("JSON object of classication added successfully and classification_report.json generated!")

JSON object of classication added successfully and classification_report.json generated!


In [ ]:
'''#To fetch the classifying items from the json file with Description in Hindi. Created for future instance.
def classify_items(json_data, org_code, sequence_to_classify): # Hindi: instead of org_code stage 3 description can be given
    # Convert JSON data to a list of dictionaries
    data = json.loads(json_data)

    if org_code.startswith("GOV"):
       data = data
       original_data = data.copy()
    else:
       # Filter data by OrgCode
       data = [item for item in data if item['OrgCode'] == org_code]
       original_data = data.copy()
       # data = [item for item in data if item['Parent'] == (item ['Code'] if item['Description'] == stage3_description)] i.e. find the code of the stage3 item description

    #print(data)

    # Initialize result dictionary
    result = {}

    text = sequence_to_classify

    for stage in range(1, 9): # Alter the stage no according to the stage 3 description
        # Get items for the current stage
        stage_items = [item for item in data if item['Stage'] == stage]
        #print(stage_items)

        # If there are no items for the current stage, break the loop
        if not stage_items:
            break

        # Get the descriptions of the items
        descriptions = [item['DescriptionHindi'] for item in stage_items] #DescriptionHindi

        # Classify the descriptions
        output, best_item, top_5_items = classification(text, descriptions)

        # Add the best item to the result
        result[f'Stage {stage}'] = {
            'Best Item': best_item,
            'Top 5 Items': top_5_items
        }
        print(best_item)
        with open('result.json', 'w') as f:
             json.dump(result, f)

        for item in data:
            if item['DescriptionHindi'] == best_item:
               best_item_code = item['Code']
               break
        print(best_item_code)
        # Update data to only include items whose parent is the code of the best item
        data = [item for item in original_data if item['Parent'] == best_item_code]
        print(data)
        print(output)

    return result'''

In [ ]:
'''import json

def add_description_hindi(json_data):
  """
  Adds a "DescriptionHindi" key with an empty string value to JSON objects that do not have it.

  Args:
    json_data: A list of JSON objects.

  Returns:
    A list of JSON objects with the "DescriptionHindi" key added.
  """

  updated_data = []
  for obj in json_data:
    if "Destination" not in obj:
      obj["Destination"] = ""
    updated_data.append(obj)

  return updated_data

# Load the JSON data from a file
with open('/content/drive/MyDrive/problem_statement_1_and_2/CategoryCode_Mapping_v2.json', 'r') as f:
  json_data = json.load(f)

# Add the "DescriptionHindi" key
updated_data = add_description_hindi(json_data)

# Save the updated JSON data to a file
with open('/content/drive/MyDrive/problem_statement_1_and_2/CategoryCode_Mapping_v2.json', 'w') as f:
  json.dump(updated_data, f)
'''